In [1]:
import pandas as pd

from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

!pip install boto3
import boto3
import json
import io
# import os
# from datetime import datetime

!pip install pyathena
from pyathena import connect


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 5.4 MB/s eta 0:00:00


In [2]:
#%% Credenciales de AmazonAthena
with open(r"/content/credenciales actualizado.txt") as f:
    creds = json.load(f)

# Cliente de S3
s3 = boto3.client(
    "s3",
    aws_access_key_id        = creds["AccessKeyId"],
    aws_secret_access_key    = creds["SecretAccessKey"],
    aws_session_token        = creds["SessionToken"],
    region_name              = creds["region_name"]
)

In [3]:
ENLACE = 'https://docs.google.com/spreadsheets/d/1PeMDOQl4Bk4DDmvHENCwa1EQq-tT0auEri0MkhPmsgY/edit?gid=0#gid=0'

# Abre el documento por su URL
sheet_url = ENLACE
spreadsheet = gc.open_by_url(sheet_url)
#Verificamos nombre de las hojas
worksheet_list = spreadsheet.worksheets()
for i, ws in enumerate(worksheet_list):
    print(f"Hoja {i+1}: {ws.title}")

# Seleccionamos la hoja 2
worksheet = worksheet_list[0]


# Convertimos a df
from gspread_dataframe import get_as_dataframe
df = get_as_dataframe(worksheet, evaluate_formulas=True)
df = df.dropna(how='all')



Hoja 1: Hoja 1
Hoja 2: Hoja 3
Hoja 3: Hoja 2


In [4]:
df

,Cierre,Ejecutivo,EJECUTIVO FINAL,DNI,Meta Utilidad,Meta MF,Meta Tasa All-In,Meta Proveedores Nuevos,Meta Stock Proveedores,Zona,Canal,Coordinador,Unnamed: 14
0,01/05/2024,GIANCARLO,GIANCARLO LOLI,70341745.0,35000.0,1000000.0,0.020,3.0,23.0,Lima,Presencial,GIANCARLO LOLI,NaN
1,01/05/2024,VIVIAN,VIVIAN PIZARRO,73195875.0,35000.0,1000000.0,0.020,3.0,23.0,Lima,Presencial,VIVIAN PIZARRO,NaN
2,01/05/2024,CARMEN,CARMEN ALVAREZ,46409463.0,60000.0,2900000.0,0.020,3.0,23.0,Lima,Presencial,VIVIAN PIZARRO,NaN
3,01/05/2024,ROSARIO,ROSARIO PEREZ,47655182.0,86800.0,4400000.0,0.020,3.0,23.0,Lima,Presencial,VIVIAN PIZARRO,NaN
4,01/05/2024,GIANINA,GIANINA POSTIGO,NaN,70000.0,2900000.0,0.020,3.0,23.0,Lima,Presencial,VIVIAN PIZARRO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,01/01/2026,RONALD,RONALD UBILLUS,44155551.0,52500.0,2000000.0,0.025,3.0,23.0,Norte,Presencial,LUIS CUNYA,NaN
593,01/01/2026,PATRICIA F.,PATRICIA FRANCO,43932962.0,105000.0,3500000.0,0.025,3.0,23.0,Norte,Presencial,PATRICIA FRANCO,NaN
594,01/01/2026,DANNY,DANNY ADANAQUE,75020504.0,30000.0,1000000.0,0.025,3.0,23.0,Norte,Presencial,PATRICIA FRANCO,NaN
595,01/01/2026,FERNANDO,FERNANDO CANO,42394775.0,90000.0,4500000.0,0.025,3.0,23.0,Sur,Presencial,FERNANDO CANO,NaN


In [5]:
df = df[['Cierre',
          'Ejecutivo',
          'EJECUTIVO FINAL',
          'DNI',
          'Meta Utilidad',
          'Meta MF',

          'Meta Tasa All-In',
          'Meta Proveedores Nuevos',
          'Meta Stock Proveedores',

          'Zona',
          'Canal',
          'Coordinador']].copy()

df['Cierre'] = pd.to_datetime(df['Cierre'], dayfirst=True).dt.date
df['DNI'] = df['DNI'].astype(str)
df['DNI'] = df['DNI'].str.replace(".0", "")
df['DNI'] = df['DNI'].str.replace("nan", "")

df.dropna(subset=['Cierre', 'Ejecutivo'], inplace = True)

df['Meta MF'] = df['Meta MF'].astype(float)
df['Meta Tasa All-In'] = df['Meta Tasa All-In'].astype(float)
df['Meta Proveedores Nuevos'] = df['Meta Proveedores Nuevos'].astype(int)
df['Meta Stock Proveedores'] = df['Meta Stock Proveedores'].astype(int)

In [6]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
################################################################################
# Hora actual en Perú (UTC-5)
now = datetime.now(ZoneInfo("America/Lima"))

# Guardar directamente el objeto datetime
df["_timestamp"] = now - timedelta(hours=5)
print(now - timedelta(hours=5))


2026-01-13 04:27:55.846401-05:00


In [7]:
!pip install pyarrow
# df.to_parquet(r'fac_metas_comercial.parquet',
#                  index = False,
#                  )


In [8]:
# # el nombre del csv, también debe ser el nombre de la carpeta
# nombre_parquet = 'fac_metas_comercial'
# nombre_carpeta = nombre_parquet

# df = pd.read_parquet(r'fac_metas_comercial.parquet')

# from datetime import datetime
# from zoneinfo import ZoneInfo

# ################################################################################
# # Hora actual en Perú (UTC-5)
# now = datetime.now(ZoneInfo("America/Lima"))
# athena_timestamp = now.strftime("%Y-%m-%d %H:%M:%S.000")
# print(athena_timestamp)
# ################################################################################
# df['_timestamp'] = athena_timestamp
# # df['_timestamp'] = pd.to_datetime(df['_timestamp'])
# df['_timestamp'] = df['_timestamp'].astype(str)

# mapping_types = {
#     'object'          : 'string',
#     'float64'         : 'double',
#     'int64'           : 'integer',
#     'bool'            : 'boolean',
#     'datetime64[ns]'  : 'date'
# }

# temp_schema = ""
# for col, dtype in df.dtypes.astype(str).to_dict().items():
#     formated_type = mapping_types[dtype]
#     temp_schema += f"""    `{col}` {formated_type},
# """

# create_table_query = f"""
# CREATE EXTERNAL TABLE prod_datalake_sandbox.ba__{nombre_parquet}
# (
# {temp_schema}

# )
# ROW FORMAT SERDE
#   'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
# STORED AS INPUTFORMAT
#   'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat'
# OUTPUTFORMAT
#   'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
# LOCATION
#     's3://prod-datalake-sandbox-730335218320/{nombre_carpeta}/'
# TBLPROPERTIES (
#   'parquet.compression'='snappy'
# )
# """
# create_table_query = create_table_query.replace("`_timestamp` string", "`_timestamp` TIMESTAMP")
# create_table_query = create_table_query.replace("`_timestamp` TIMESTAMP,", "`_timestamp` TIMESTAMP")
# print(create_table_query)

In [9]:
#%%
nombre_tabla = 'fac_metas_comercial'

# Cliente de S3
s3 = boto3.client(
    "s3",
    aws_access_key_id     = creds["AccessKeyId"],
    aws_secret_access_key = creds["SecretAccessKey"],
    aws_session_token     = creds["SessionToken"],
    region_name           = creds["region_name"]
)

# ==== CONFIGURACIÓN ====
bucket_name = "prod-datalake-sandbox-730335218320"
s3_prefix   = f"{nombre_tabla}/"  # carpeta lógica en el bucket

# ==== EXPORTAR A PARQUET EN MEMORIA ====
parquet_buffer = io.BytesIO()
df.to_parquet(parquet_buffer, index=False, engine="pyarrow")
# también puedes usar engine="fastparquet" si lo prefieres

# Nombre de archivo con timestamp (opcional)
s3_key = f"{s3_prefix}{nombre_tabla}.parquet"

# Subir directamente desde el buffer
s3.put_object(
    Bucket = bucket_name,
    Key    = s3_key,
    Body   = parquet_buffer.getvalue()
)

print(f"✅ Archivo subido a s3://{bucket_name}{s3_key}")
print('se puede ubicar con el nombre prod_datalake_sandbox.ba__fac_metas_comercial')

✅ Archivo subido a s3://prod-datalake-sandbox-730335218320fac_metas_comercial/fac_metas_comercial.parquet
se puede ubicar con el nombre prod_datalake_sandbox.ba__fac_metas_comercial
